<a href="https://colab.research.google.com/github/earandir/DataAnalysis/blob/main/Modulo03/Sesion08/M03S08R02tablas_a_dataframes_sqlite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejemplo 2: Consultas a la base de datos y construcción de `DataFrames`

### 1. Objetivos:
    - Realizar consultas a la base de datos y construir un `DataFrame` para cada tabla.
 
---
    
### 2. Desarrollo:

#### a) Tablas a `DataFrames`

In [1]:
import pandas as pd
import sqlite3

Realizamos la conexión y obtenemos nuestro índice (cursor) para poder realizar operaciones con la base de datos:

In [2]:
bd = "movielens.sqlite3"
conn = sqlite3.connect(bd)
cur = conn.cursor()

cur

Ahora vamos a obtener la lista de todos los registros de la tabla `users` usando la función `execute(-sql-)` y la instrucción SQL:

`SELECT * FROM users`

In [3]:
sql = 'SELECT * FROM users;'

cur.execute(sql)

Ahora hay que obtener todos los resultados de la consulta con:
    
`mi_cursor.fetchall()`

In [4]:
resultados = cur.fetchall()



Si recuerdas el último ejemplo, `fetchall` nos regresa una `lista` de `tuplas`. Cada `tupla` representa una fila de nuestro conjunto de datos, por ejemplo obtén el primer registro se podría obtener como:

In [5]:
resultados[0]

(1, 'F', 1, 10, '48067')

O podríamos obtener el género del primer usuario con:

In [6]:
resultados[0][1]

'F'

Algo muy util es obtener la lista de los nombres de las columnas de la tabla consultada, para eso se hacer uso del atributo:

`mi_cursor.description`

In [7]:
cur.description

(('user_id', None, None, None, None, None, None),
 ('gender', None, None, None, None, None, None),
 ('age', None, None, None, None, None, None),
 ('occupation', None, None, None, None, None, None),
 ('cp', None, None, None, None, None, None))

y aplicando listas de compresión se puede obtener la lista deseada:

In [8]:
#first_tuple_elements = []
#for a_tuple in cur.description:
#  first_tuple_elements.append(a_tuple[0])

columnas = [a_tuple[0] for a_tuple in cur.description]
columnas

['user_id', 'gender', 'age', 'occupation', 'cp']

In [9]:
cols = [t[0] for t in cur.description]

cols

['user_id', 'gender', 'age', 'occupation', 'cp']

Afortunadamente `pandas` puede recibir justamente `listas` de `tuplas` como ingredientes para construir `DataFrames`. Sólo hace falta indicarle el nombre de las columnas. Los nombres de las columnas están especificados en el [archivo Readme.md](../../Datasets/MovieLens/Readme.md) que venía incluido con el dataset:

Así que ahora se crea el dataframe usando la forma:

`pd.DataFrame(-lista de registros-,
    columns=-lista de nombres de columnas-)`

In [10]:
df = pd.DataFrame(resultados, columns=columnas)

df.head()

,user_id,gender,age,occupation,cp
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


Otra manera de realizar esta operación de una manera más compacta es utilizando la siguiente función de Pandas:

`pd.read_sql(-consulta sql-, -variable de conexión a la BD-)`

así que vamos a obtener justo el mismo dataframe y lu guardaremos en la viable `df_1`:

In [11]:
df_1 = pd.read_sql(sql, conn)

df_1.head()

,user_id,gender,age,occupation,cp
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


Sería una buena idea convertir la columna `user_id` en índice para no tener información redundante usando:

`dataframe.set_index(-columna-, drop=True)`

y el resultado lo guardamos en la variable `df_2`:

In [12]:
df_2 = df_1.set_index(columnas, drop=True)

df_2.head(3)

,,,,
user_id,gender,age,occupation,cp
1,F,1,10,48067
2,M,56,16,70072
3,M,25,15,55117


¡Listo! Vamos a guardar nuestros `DataFrames` en el archivo `users.csv` para que no haga falta volver a extraerlos de la base de datos. Ya podemos decirle adiós a nuestro fiel MySQL. ¡Chao, bellisimo!

In [13]:
df_2.to_csv('users.csv')

Ahora hagamos lo mismo para la tabla `occupations`, obtengamos todos los registros, examinemos, obtengamos los nombres de las columnas, creemos el dataframe, reasignemos índices en caso de ser necesario y guardemos el resultado en el archivo `occupations.csv`

In [14]:
sql_occupation = 'SELECT * FROM occupations'

df_occupation = pd.read_sql(sql_occupation, conn)

df_occupation.set_index(list(df_occupation.columns), drop=True, inplace=True)

df_occupation

,
occupation_id,description
0,other or not specified
1,academic/educator
2,artist
3,clerical/admin
4,college/grad student
...,...
16,self-employed
17,technician/engineer
18,tradesman/craftsman


Haaa y no olvides cerrar tu índice y tu conexión:

In [15]:
cur.close()
conn.close()

---
---

## Reto 2: Convertir tablas en MySQL a `DataFrames` de `pandas`

### 1. Objetivos:
    - Solicitar todos los datos de las tablas que están almacenadas en nuestra base de datos, convertirlos a `DataFrames` y guardarlos.
    
---
    
### 2. Desarrollo:

#### a) Tablas a `DataFrames`

Ya que tenemos nuestra conexión funcionando adecuadamente, vamos a utilizarla para realizar consultas a las base de datos y construir una base de datos local. Tu Reto consiste en los siguientes pasos:

1. Vuelve a establecer la conexión a la base de datos
2. Usando el comando `SELECT * FROM nombre_de_tabla`, realiza consultas a cada una de las 5 tablas que existen en la base de datos.
3. Crea un `DataFrame` por cada tabla que obtuviste. Para asignarle los nombres de las columnas correctamente, revisa el archivo [Readme.md](../../Datasets/MovieLens/Readme.md) donde está contenida toda esa información.
4. Asegúrate de que el índice sea adecuado en cada `DataFrame`. En los casos en los que haya datos redundantes, convierte una de las columnas en índice.
5. Si lo deseas ordena las columnas de la manera en la que mejor te parezca.
6. Guarda tus `DataFrames` en formato .csv para utilizarlos en los siguientes Retos.

> **NOTA IMPORTANTE**: La tabla movies es un poco complicada porque contiene muchos signos distintos. Tanto en la columna de nombre de película como la de género, encontramos signos como `,`, `:`, `.`, `|`. Esto hace un poco complicado el almacenamiento y lectura de este archivo. Si elijes guardar este archivo como un .csv separado por comas (`,`), a la hora de leerlo de regreso, `pandas` puede confundirse y pensar que el título de una película que contiene comas constituye dos columnas. Por esta razón, te recomiendo que la tabla `movies` la guardes agregando un separador poco convencional como `sep='$'`. De esta manera será muchísimo más fácil leer tu archivo de regreso usando ese separador.

In [16]:
import pandas as pd
import sqlite3

In [17]:
bd = "movielens.sqlite3"
conn = sqlite3.connect(bd)


In [18]:
users_sql = 'SELECT * FROM users'
occupations_sql = 'SELECT * FROM occupations'
ratings_sql = 'SELECT * FROM ratings'
age_ranges_sql = 'SELECT * FROM age_ranges'
movies_sql = 'SELECT * FROM movies'

tables = 'SELECT name FROM sqlite_master WHERE type=\'table\''


In [19]:
def get_dataframe(a_sql, a_conn):
  a_df = pd.read_sql(a_sql, a_conn)
  a_df.set_index(list(a_df.columns), drop=True, inplace=True)
  return a_df

In [20]:
df_users = get_dataframe(users_sql, conn)

df_occupations = get_dataframe(occupations_sql, conn)

df_ratings = get_dataframe(ratings_sql, conn)

df_age_ranges = get_dataframe(age_ranges_sql, conn)

df_movies = get_dataframe(movies_sql, conn)

In [21]:
df_users.to_csv('users.csv')

df_occupations.to_csv('occupations.csv')

df_ratings.to_csv('ratings.csv')

df_age_ranges.to_csv('age_ranges.csv')

df_movies.to_csv('movies.csv')


In [22]:
conn.close()

Compara con tus compañeros y revisa con la experta para que todos estén seguros de que tienen sus `DataFrames` estructurados de la manera correcta y que sus archivos .csv fueron creados exitosamente. Vamos a utilizar estos archivos en los Retos siguientes, así que es muy importante que tus datos estén estructurados adecuadamente.